# Smart-G demo notebook , part 2</b>

This is an interactive document allowing to run Smart-G with python and visualize the results. <br>
*Tips*: cells can be executed with shift-enter. Tooltips can be obtained with shift-tab. More information [here](http://ipython.org/notebook.html) or in the help menu. [A table of content can also be added](https://github.com/minrk/ipython_extensions#table-of-contents).

In [ ]:
%pylab inline
# next 2 lines allow to automatically reload modules that have been changed externally
%reload_ext autoreload
%autoreload 2
from __future__ import absolute_import, division, print_function
import os, sys
sys.path.insert(0, os.path.dirname(os.getcwd()))
from smartg.smartg import Smartg
from smartg.smartg import RoughSurface, LambSurface, FlatSurface, Environment, Albedo_cst
from smartg.atmosphere import AtmAFGL, AeroOPAC, CloudOPAC, diff1, read_phase
from smartg.water import IOP_1, IOP
from smartg.reptran import REPTRAN, reduce_reptran
from smartg.tools.tools import SpherIrr, Irr, reduce_Irr
from smartg.tools.luts import LUT, MLUT, Idx, merge, read_mlut
from smartg.tools.smartg_view import spectrum_view, plot_polar, smartg_view, input_view, mdesc, transect_view, profile_view, phase_view
from ipywidgets import interact, interact_manual
import hapi_gpu as hapi
from matplotlib import colors
colors_ = list(six.iteritems(colors.cnames))
# Add the single letter colors.
for name, rgb in six.iteritems(colors.ColorConverter.colors):
    hex_ = colors.rgb2hex(rgb)
    colors_.append((name, hex_))

# Transform to hex color values.
hex_ = [color[1] for color in colors_]
# Get the rgb equivalent.
rgb = [colors.hex2color(color) for color in hex_]

# Split the hsv values to sort.
rr = [color[0] for color in rgb]
gg = [color[1] for color in rgb]
bb = [color[2] for color in rgb]

# Sort by hue, saturation and value.
ind = np.lexsort((rr, gg, bb))
sorted_colors = [colors_[i] for i in ind]

# Equivalence Theorem

In [ ]:
%%time
# Here we compare the TOA radiance simulated with
# plane parallel and spherical atmospheres, at 443 nm.
atm=AtmAFGL('afglt',grid=np.linspace(100.,0.,num=100), O3=400., NO2=False)
results = []
le={}
le.update(th=np.linspace(0.,89.,num=24)*np.pi/180)
le.update(phi=np.array([0.],dtype=np.float32)*np.pi/180)
for pp in [True, False]:
    S=Smartg(pp=pp, double=True)
    for BEER in [0, 1]:
        # Absorption is computed with Beer Law (BEER=1) or Single scattering albedo (BEER=0)
        results.append(S.run(THVDEG=0., wl=550., NBPHOTONS=1e8, le=le, atm=atm, BEER=BEER))
    
SSA_PP = results[0]['I_up (TOA)']
ET_PP  = results[1]['I_up (TOA)']
SSA_SP = results[2]['I_up (TOA)']
ET_SP  = results[3]['I_up (TOA)']

In [ ]:
((ET_SP-ET_PP)/ET_SP*100.).sub()[0,:].plot(fmt='k-o',vmin=-2,vmax=2)
((SSA_SP-SSA_PP)/SSA_SP*100.).sub()[0,:].plot(fmt='r-x',vmin=-2,vmax=2)
figure()
((SSA_SP-ET_SP)/SSA_SP*100.).sub()[0,:].plot(fmt='r-o')
((SSA_PP-ET_PP)/SSA_PP*100.).sub()[0,:].plot(fmt='k-o',vmin=-1,vmax=1)

# ALIS

## Pure scattering

In [ ]:
S0= Smartg(alis=None        , double=True, pp=True)
S = Smartg(alis={'nlow':11} , double=True, pp=True)

In [ ]:
%%time
dl  = 0.2
lmin= 320.
lmax= 340.
wref= (lmax-lmin)/2 + lmin
NW  = int((lmax-lmin)/dl) + 1
wl  = np.linspace(lmin,lmax,num=NW)
NL  = 20
aer=AeroOPAC('maritime_tropical',0.1, 410.)
comp=[aer]
znew =np.linspace(100.,0.,num=NL+1)
pfwav=[410.]
atm=AtmAFGL('afglus', grid=znew , O3=400., NO2=True, comp=comp, pfwav=pfwav)
surf=None
le={}
le.update(th=np.array([40.])*np.pi/180)
le.update(phi=np.array([120.])*np.pi/180)
NB=1e7
mle = S.run( THVDEG=30., wl=wl, le=le, surf=surf, NBPHOTONS=NB, atm=atm, progress=True)
m0le= S0.run(THVDEG=30., wl=wl, le=le, surf=surf, NBPHOTONS=NB*5, atm=atm, progress=True)
print (mle.attrs['kernel time (s)'])

In [ ]:
f=spectrum_view(mle.sub({ 'Azimuth angles':0, 'Zenith angles':0}), QU=True, fmt='-', color='r')
_=spectrum_view(m0le.sub({'Azimuth angles':0, 'Zenith angles':0}), QU=True, fmt='.', color='b', fig=f)
figure()
(100*(mle['I_up (TOA)']-m0le['I_up (TOA)'])/mle['I_up (TOA)']).sub()[:,0,0].plot(fmt='.-k')

## HITRAN

In [ ]:
hapi.db_begin('data')

## Jacobians

### Sensitivity of O2A to surface pressure

In [ ]:
%%time
dl= 0.0100
lmin=759.
lmax=769.001
NW=int((lmax-lmin)/dl)
vmin=1e7/lmax
vmax=1e7/lmin
dv=(vmax-vmin)/NW
#vmin=12980. #cm-1
#vmax=13200.25 #cm-1
hapi.fetch('O2i1',7,1,vmin-100,vmax+100)
hapi.fetch('O2i2',7,2,vmin-100,vmax+100)
hapi.fetch('O2i3',7,3,vmin-100,vmax+100)
#dv=.25 #cm-1
NL=21
#NW= int((vmax-vmin)/dv)
print ('NW: ', NW)
pmin=0.001
pmax=1012.
pnew=np.linspace(pmin,pmax, num=NL)
#znew=abs(-8. *np.log(pnew/pmax))
znew =np.linspace(100.,0.,num=NL)

NB=1e7

P0l=[1000., 1010.]

ao2=np.zeros((NW,NL),dtype=float)

for i,P0 in enumerate(P0l):
    j=0
    atm = AtmAFGL('afglmw', grid=znew, NO2=False, O3=0., P0=P0)
    
    for p,t,o2,z in zip(atm.prof.P,atm.prof.T,atm.prof.dens_o2,atm.prof.z):
        nuo2,coefo2 = hapi.absorptionCoefficient_Voigt_gpu(SourceTables=['O2i1','O2i2','O2i3'],HITRAN_units=True,
            OmegaRange=[vmin,vmax],OmegaStep=dv,GammaL='gamma_self',
            Environment={'p':p/1013.,'T':t})
        ao2[:,j] = coefo2 * o2 * 1e5
        j=j+1
    wl=(1e7/nuo2)
    wl=wl[::1]

    pro = AtmAFGL('afglmw', grid=znew, NO2=False, O3=0., P0=P0).calc(wl)
    if i!=0 :      
        ab  = np.concatenate((ab , ao2), axis=0)
        ray = np.concatenate((ray, pro['OD_r'].data), axis=0)
        
    else: 
        ab = ao2
        ray= pro['OD_r'].data

wls = np.concatenate((wl,wl))
print (ao2.max())
atm = AtmAFGL('afglmw', grid=znew, prof_ray=ray, prof_abs=ab)

In [ ]:
S0=Smartg(alis={'nlow':3}, double=True, pp=True)
tv=np.linspace(0.,75.,num=6)
le={}
le.update(th=tv*np.pi/180)
le.update(phi=np.linspace(0.,360.,num=1)*np.pi/180)
surf=RoughSurface(SUR=1)
surf=None

In [ ]:
%%time
m=S0.run(THVDEG=30., wl=wls, atm=atm, surf=surf, le=le, NBPHOTONS=1e5, progress=False)

In [ ]:
field='I_up (TOA)'
I=m[field].sub(d={'wavelength':arange(NW)})
I.names[0]=r'$\lambda (nm)$'
I.axes[0]=wl
I.desc=mdesc(field)
for t in tv[:-1]:
    _= I.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.04, vmin=0, label='%.0f'%t)
legend()

figure()
jac= (m[field].sub(d={'wavelength':arange(NW)+NW}) - 
      m[field].sub(d={'wavelength':arange(NW)}) ) / (P0l[-1]-P0l[0]) 
jac.names[0]=r'$\lambda (nm)$'
jac.axes[0]=wl
jac.desc=r'$d$'+mdesc(field)+r'$/dP_0$' + '  ' + r'$(hPa^{-1})$'

for t in tv[:-1]:
    _=jac.sub()[:,0,Idx(t)].plot(fmt='-', vmax=2.e-4, vmin=0, label='%.0f'%t)
legend()

figure()
field='Q_up (TOA)'
I=m[field].sub(d={'wavelength':arange(NW)})
I.names[0]=r'$\lambda (nm)$'
I.axes[0]=wl
I.desc=mdesc(field)
for t in tv[:-1]:
    _= I.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.001, vmin=-0.005, label='%.0f'%t)
#legend()

figure()
jac= (m[field].sub(d={'wavelength':arange(NW)+NW}) - 
      m[field].sub(d={'wavelength':arange(NW)}) ) / (P0l[-1]-P0l[0]) 
jac.names[0]=r'$\lambda (nm)$'
jac.axes[0]=wl
jac.desc=r'$d$'+mdesc(field)+r'$/dP_0$' + '  ' + r'$(hPa^{-1})$'

for t in tv[:-1]:
    _=jac.sub()[:,0,Idx(t)].plot(fmt='-', vmax=0.5e-5, vmin=-3.e-5, label='%.0f'%t)
#legend(loc='best')

## Emde et al 2010

### Rayleigh  and black surface

In [ ]:
S=Smartg(alis={'nlow':3}, double=True, pp=True)

In [ ]:
fgas = '/home/mathieu/work/_archive/MCCuda/artdeco_output/cTauGas_ray_O2.dat'
gas_valid = diff1(np.loadtxt(fgas, skiprows=7)[:,1:].T, axis=1)
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/_archive/MCCuda/artdeco_output/cTauRay_ray_O2.dat'
ray_valid = diff1(np.loadtxt(fray, skiprows=7)[:,1:].T, axis=1)
data_valid=np.loadtxt('/home/mathieu/work/_archive/MCCuda/resultat/artdeco_lbl_nstr_32_ray_O2.dat')

In [ ]:
atm_valid = AtmAFGL('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=[760.], 
                    prof_ray= ray_valid, prof_abs= gas_valid)

In [ ]:
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
mle_valid5=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=3e7, atm=atm_valid, 
               NBLOOP=1e5, progress=True, RTER=6370.)
print (mle_valid5.attrs['kernel time (s)'])

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid5['Q_up (TOA)']*mle_valid5['Q_up (TOA)']+mle_valid5['U_up (TOA)']*mle_valid5['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))
i=mle_valid5['I_up (TOA)']
##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.01)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=3, vmin=-3)
##
figure()
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='g-', vmax=0.005)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=3, vmin=-3)

### Rayleigh , aerosol and black surface

In [ ]:
S=Smartg(alis={'nlow':3}, double=True, pp=True)

In [ ]:
fgas = '/home/mathieu/work/_archive/MCCuda/artdeco_output/cTauGas_ray_desert_O2.dat'
gas_valid = diff1(np.loadtxt(fgas, skiprows=7)[:,1:].T, axis=1)
z_valid   = np.loadtxt(fgas, skiprows=7)[:,0]
w_valid   = np.array(open(fgas).readlines()[5].split()).astype(float)
fray = '/home/mathieu/work/_archive/MCCuda/artdeco_output/cTauRay_ray_desert_O2.dat'
ray_valid = diff1(np.loadtxt(fray, skiprows=7)[:,1:].T, axis=1)
faer_abs = '/home/mathieu/work/_archive/MCCuda/artdeco_output/cTauAbs_ptcle_ray_desert_O2.dat'
aer_abs_valid = diff1(np.loadtxt(faer_abs, skiprows=7)[:,1:].T, axis=1)
faer_sca = '/home/mathieu/work/_archive/MCCuda/artdeco_output/cTauSca_ptcle_ray_desert_O2.dat'
aer_sca_valid = diff1(np.loadtxt(faer_sca, skiprows=7)[:,1:].T, axis=1)

# aerosols phase matrix import
faer_phase= '/home/mathieu/work/_archive/MCCuda/artdeco_output/phasemat_ray_desert_O2.dat'
f=open(faer_phase,'r')
N=np.genfromtxt(faer_phase, usecols=range(1), max_rows=1, dtype=int)
pfwav=[]
P=[]
Npf=3
data=np.zeros((Npf,1,N,5), dtype=float32) 
for k in range(Npf):
    pfwav.append(np.genfromtxt(faer_phase, usecols=range(1), skip_header=(1+(2+N)*k), max_rows=1))
    pizero=np.genfromtxt(faer_phase, usecols=range(1), skip_header=(1+(2+N)*k+1), max_rows=1)
    data[k,0,:,:] = np.genfromtxt(faer_phase, usecols=range(5), skip_header=(1+(2+N)*k+2), max_rows=N)
    
data=data.swapaxes(2,3)
phase_valid = LUT(data[:,:,1:,:],
          names = ['wav_phase_atm', 'z_phase_atm', 'stk','theta'] ,
          axes  = [pfwav, [0], None, data[0,0,0,:]])
   
data_valid=np.loadtxt('/home/mathieu/work/_archive/MCCuda/resultat/artdeco_lbl_nstr_32_ray_desert_O2.dat')

In [ ]:
aer_ext_valid  = aer_sca_valid + aer_abs_valid
aer_ssa_valid  = aer_sca_valid / aer_ext_valid
aer_ssa_valid[aer_ext_valid==0]=1.

comp=[AeroOPAC('desert',0.5, 550., phase= phase_valid)]
atm_valid = AtmAFGL('afglmw', grid=z_valid, O3=0., NO2=False, pfwav=pfwav, comp=comp,
                    prof_ray= ray_valid,
                    prof_aer= (aer_ext_valid,aer_ssa_valid),
                    prof_abs= gas_valid
                    )

In [ ]:
le={}
le.update(th=np.array([20.])*np.pi/180)
le.update(phi=np.array([180.])*np.pi/180)
mle_valid5=  S.run(THVDEG=30., wl=w_valid, le=le,  DEPO=0., NBPHOTONS=1e8, atm=atm_valid, 
                   NBLOOP=10000, progress=False, RTER=63700.)
print (mle_valid5.attrs['kernel time (s)'])

In [ ]:
i_valid=data_valid[:,1]
ip_valid=np.sqrt(data_valid[:,2]*data_valid[:,2]+data_valid[:,3]*data_valid[:,3])
ip=mle_valid5['Q_up (TOA)']*mle_valid5['Q_up (TOA)']+mle_valid5['U_up (TOA)']*mle_valid5['U_up (TOA)']
ip=ip.apply(np.sqrt,mdesc('Ip_up (TOA)'))
i=mle_valid5['I_up (TOA)']
##
plot(w_valid,  i_valid, 'r')
f= i.sub()[:,0,0].plot(fmt='g-', vmax=0.01)
figure()
df=i.sub()[:,0,0]- i_valid
dff=df/i_valid*100
dff.plot(fmt='k-', vmax=10, vmin=-6)
##
figure()
plot(w_valid,  ip_valid, 'r')
f= ip.sub()[:,0,0].plot(fmt='g-', vmax=0.005)
figure()
df=ip.sub()[:,0,0]- ip_valid
dff=df/ip_valid*100
dff.plot(fmt='k-', vmax=10, vmin=-10)